In [125]:
import numpy as np
from torchvision.datasets import CIFAR100
from collections import defaultdict
import random
import pickle
from tqdm import tqdm
import torch
import torch.nn as nn

In [126]:
# Get Dataset
dataset = CIFAR100(root='./data', train=True, download=True)
print(dataset)

with open('data/cifar-100-python/meta', 'rb') as fo:
    metadata = pickle.load(fo, encoding='bytes')

classLabels = dict(list(enumerate(metadata[b'fine_label_names'])))
print(classLabels)

Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
{0: b'apple', 1: b'aquarium_fish', 2: b'baby', 3: b'bear', 4: b'beaver', 5: b'bed', 6: b'bee', 7: b'beetle', 8: b'bicycle', 9: b'bottle', 10: b'bowl', 11: b'boy', 12: b'bridge', 13: b'bus', 14: b'butterfly', 15: b'camel', 16: b'can', 17: b'castle', 18: b'caterpillar', 19: b'cattle', 20: b'chair', 21: b'chimpanzee', 22: b'clock', 23: b'cloud', 24: b'cockroach', 25: b'couch', 26: b'crab', 27: b'crocodile', 28: b'cup', 29: b'dinosaur', 30: b'dolphin', 31: b'elephant', 32: b'flatfish', 33: b'forest', 34: b'fox', 35: b'girl', 36: b'hamster', 37: b'house', 38: b'kangaroo', 39: b'keyboard', 40: b'lamp', 41: b'lawn_mower', 42: b'leopard', 43: b'lion', 44: b'lizard', 45: b'lobster', 46: b'man', 47: b'maple_tree', 48: b'motorcycle', 49: b'mountain', 50: b'mouse', 51: b'mushroom', 52: b'oak_tree', 53: b'orange', 54: b'orchid', 55: b'otter', 56: b'palm_tree', 57: b'pear

In [127]:
with open('data/cifar-100-python/train', 'rb') as fo:
    trainMeta = pickle.load(fo, encoding='bytes')

with open('data/cifar-100-python/test', 'rb') as fo:
    testMeta = pickle.load(fo, encoding='bytes')


totalTest = len(testMeta[b'data'])
valNum = 4000

# Get training/testing data and labels
trainData = trainMeta[b'data']
trainData = trainData.reshape((len(trainData), 3, 32, 32))
trainLabel = np.array(trainMeta[b'fine_labels'])

mask = list(range(valNum))
valData = testMeta[b'data'][mask]
valData = valData.reshape((len(valData), 3, 32, 32))
valLabel = np.array(trainMeta[b'fine_labels'])[mask]

mask = list(range(valNum, totalTest))
testData = testMeta[b'data'][mask]
testData = testData.reshape((len(testData), 3, 32, 32))
testLabel = np.array(testMeta[b'fine_labels'])[mask]

In [128]:
# Get the number of images per class
imgPerClass = []

for cls in range(100):
    num = 500 * (0.05 ** (cls / 99))
    imgPerClass.append(int(num))

print(imgPerClass)
print(np.sum(imgPerClass))

[500, 485, 470, 456, 442, 429, 416, 404, 392, 380, 369, 358, 347, 337, 327, 317, 308, 298, 290, 281, 272, 264, 256, 249, 241, 234, 227, 220, 214, 207, 201, 195, 189, 184, 178, 173, 168, 163, 158, 153, 149, 144, 140, 136, 132, 128, 124, 120, 116, 113, 110, 106, 103, 100, 97, 94, 91, 89, 86, 83, 81, 78, 76, 74, 72, 69, 67, 65, 63, 61, 60, 58, 56, 54, 53, 51, 50, 48, 47, 45, 44, 43, 41, 40, 39, 38, 37, 35, 34, 33, 32, 31, 30, 29, 29, 28, 27, 26, 25, 25]
15907


In [129]:
# Get LT training data
trainDataLT, trainLabelLT = [], []
random.shuffle(imgPerClass)

for cls, numImg in enumerate(imgPerClass):
    clsIndx = np.where(trainLabel == cls)[0]
    numSampledImages = np.random.choice(clsIndx, numImg, replace=False)

    trainDataLT.append(trainData[numSampledImages])
    trainLabelLT.append(trainLabel[numSampledImages])

trainDataLT, trainLabelLT = np.concatenate(trainDataLT), np.concatenate(trainLabelLT)

In [ ]:

def test_model(model, dataSet):

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_data in dataSet:
            images, labels = batch_data

            output = model(images)
            predicted = torch.argmax(output, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = correct // total
    return acc

In [ ]:
def train_model(trainData, valData, epochs: int, model=None, optimizer=None, scheduler=None, lossFunc=None, freq=50) -> None:

    for epoch in range(epochs):
        model.train()
        
        avgLoss = 0
        totalCorrect = 0
        totalNum = 0

        for i, batch in tqdm(enumerate(trainData)):
            data, labels = batch

            optimizer.zero_grad()
            output = model(data)
            target = torch.tensor(labels, dtype=torch.long)
            loss = lossFunc(output, target)
            loss.backward()
            optimizer.step()

            avgLoss += loss.item()
            pred = torch.argmax(output, 1)
            totalCorrect += (pred == labels).sum().item()
            totalNum += labels.size(0)

            if i % freq == 0:    # print every certain number of mini-batches
                avgLoss = avgLoss / freq
                accuracy = totalCorrect / totalNum
                last_lr = scheduler.get_last_lr()[0]
                print(f'[{epoch + 1}/{epochs}, {i + 1:5d}/{len(trainData)}] loss: {avgLoss:.3f} acc: {accuracy:.3f} lr: {last_lr:.5f}')
                avgLoss = 0.0
                totalCorrect = 0.0
                totalNum = 0.0

        scheduler.step()

        val_acc = test_model(model, valData)
        print(f'[{epoch + 1}/{epochs}] val acc: {val_acc:.3f}')

            


In [159]:
class baseModel(nn.Module):
    def __init__(self):
        super(baseModel, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 100)

        self.dropout = nn.Dropout(0.5)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))

        x = torch.flatten(x, 1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.softmax(self.fc2(x))

        return x

In [160]:
trainSet = torch.utils.data.TensorDataset(torch.Tensor(trainDataLT), torch.Tensor(trainLabelLT))
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=32, shuffle=True, num_workers=2)
valSet = torch.utils.data.TensorDataset(torch.Tensor(valData), torch.Tensor(valLabel))
valLoader = torch.utils.data.DataLoader(valSet, batch_size=32, shuffle=True, num_workers=2)

model = baseModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss = nn.CrossEntropyLoss()

train_model(trainLoader, valLoader, 10, model, optimizer, scheduler, loss)

0it [00:00, ?it/s]c:\Users\Bob\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\Bob\AppData\Local\Temp\ipykernel_33592\1618133607.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(labels, dtype=torch.long)
11it [00:02,  7.46it/s]

[1/10,     1/498] loss: 0.092 acc: 0.000 lr: 0.00100


55it [00:03, 44.20it/s]

[1/10,    51/498] loss: 4.610 acc: 1.125 lr: 0.00100


109it [00:04, 52.03it/s]

...

[10/10,   401/498] loss: 4.610 acc: 1.250 lr: 0.00100


456it [00:10, 53.21it/s]

[10/10,   451/498] loss: 4.610 acc: 1.250 lr: 0.00100


498it [00:12, 41.47it/s]


[10/10] val acc: 1.000
